In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
!wget https://raw.githubusercontent.com/infosecdemos/ml-2020/master/ml-ids/datasets/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.zip -O dataset.zip
!unzip -u dataset.zip
df = pd.read_csv('Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', engine='python')

--2020-09-28 08:53:21--  https://raw.githubusercontent.com/infosecdemos/ml-2020/master/ml-ids/datasets/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15190336 (14M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  14.49M  30.3MB/s    in 0.5s    

2020-09-28 08:53:21 (30.3 MB/s) - ‘dataset.zip’ saved [15190336/15190336]

Archive:  dataset.zip


In [5]:
df.shape

(458968, 85)

In [6]:
# https://pair-code.github.io/facets/
!pip install facets-overview

In [11]:
!wget https://raw.githubusercontent.com/infosecdemos/ml-2020/master/ml-ids/datasets/web_attacks_balanced.zip -O web_attacks_balanced.zip
!unzip -u web_attacks_balanced.zip
df = pd.read_csv('web_attacks_balanced.csv')

--2020-09-28 09:10:25--  https://raw.githubusercontent.com/infosecdemos/ml-2020/master/ml-ids/datasets/web_attacks_balanced.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 768997 (751K) [application/zip]
Saving to: ‘web_attacks_balanced.zip’

web_attacks_balance 100%[===================>] 750.97K  --.-KB/s    in 0.1s    

2020-09-28 09:10:27 (6.52 MB/s) - ‘web_attacks_balanced.zip’ saved [768997/768997]

Archive:  web_attacks_balanced.zip
  inflating: web_attacks_balanced.csv  


In [12]:
# Create the feature stats for the datasets and stringify it.
import base64
from facets_overview.generic_feature_statistics_generator import GenericFeatureStatisticsGenerator

gfsg = GenericFeatureStatisticsGenerator()
proto = gfsg.ProtoFromDataFrames([{'name': 'train + test', 'table': df}])
protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")

In [13]:
# Display the facets overview visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """
        <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html" >
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display(HTML(html))